In [1]:
# import relevant packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# read in data
df = pd.read_csv('../data/1-uw_layer1_data.csv')

# display
df.head()

,BOREHOLE_ID,BOREHOLE_NAME,BOREHOLE_TYPE,BOREHOLE_DEPTH_FT,ELEVATION_FT,LATITUDE,LONGITUDE,LAYER_NUMBER,TOP_DEPTH_FT,BOTTOM_DEPTH_FT,USCS,SIMPLE_USCS,LAYER_THICKNESS_FT
0,7138,B-1,Geotechnical,20.3,167.8,47.656719,-122.305728,1,0.0,1.0,SP,S,1.0
1,7143,B-1-92,Geotechnical,48.5,123.2,47.653642,-122.306837,1,0.0,1.0,ML,M,1.0
2,7144,B-2-92,Geotechnical,42.5,122.9,47.653766,-122.306468,1,0.0,1.0,ML,M,1.0
3,7145,B-3-92,Geotechnical,43.0,117.3,47.653256,-122.306638,1,0.0,1.0,ML,M,1.0
4,7146,B-4-92,Geotechnical,43.0,123.4,47.653709,-122.306259,1,0.0,0.5,SM,S,0.5


In [90]:
ucsc_classes = df['USCS']

In [91]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(list(uscs_classes))
le.classes_

array(['CH', 'CL', 'GM', 'GP', 'GP-GM', 'GW', 'MH', 'ML', 'OH', 'OL',
       'PT', 'SC', 'SM', 'SP', 'SP-SM', 'SW', 'SW-SM'], dtype='<U5')

In [92]:
len(uscs_classes)

506

In [93]:
np.unique(le.transform(uscs_classes))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

In [94]:
parent = {} # empty dict for classes
for i in np.unique(le.transform(uscs_classes)):
    parent[i] = le.inverse_transform([i])[0][0]

In [95]:
parent.values()

dict_values(['C', 'C', 'G', 'G', 'G', 'G', 'M', 'M', 'O', 'O', 'P', 'S', 'S', 'S', 'S', 'S', 'S'])

In [96]:
root = {} # empty dict for parent classes
root['G'] = 'Course'
root['S'] = 'Course'
root['M'] = 'Fine'
root['C'] = 'Fine'
root['O'] = 'Fine'
root['P'] = 'Organic'

root

{'G': 'Course',
 'S': 'Course',
 'M': 'Fine',
 'C': 'Fine',
 'O': 'Fine',
 'P': 'Organic'}

In [97]:
parent

{0: 'C',
 1: 'C',
 2: 'G',
 3: 'G',
 4: 'G',
 5: 'G',
 6: 'M',
 7: 'M',
 8: 'O',
 9: 'O',
 10: 'P',
 11: 'S',
 12: 'S',
 13: 'S',
 14: 'S',
 15: 'S',
 16: 'S'}

In [99]:
def loss(ytrue, ypred):
    error = 0
    ycheck = ytrue - ypred

    sim = 1 # similarity factor
    for i, yc in enumerate(ycheck):
        # if zero, the classes are matching and 0 
        # is added to the total error
        ptrue, ppred = parent[ytrue[i]], parent[ypred[i]]
        rtrue, rpred = root[ptrue], root[ppred]
        if yc == 0:
            sim = 1
        elif ptrue == ppred:
            sim = 0.5
        elif rtrue == rpred:
            sim = 0.25
        else:
            sim = 0
        error += (1-sim)
        
    return error

In [ ]:
# testing!

In [100]:
ypred = np.random.randint(0, 16, size=500)
ytrue = np.random.randint(0, 16, size=500)

In [101]:
loss(ytrue, ypred)

404.5

In [102]:
ypred = ytrue
loss(ytrue, ypred) # should be zero

0